In [ ]:
import csv
import pandas as pd
import numpy as np
import os
import arff
import re
import codecs
path_to_dataset_folder = 'C:/Users/Dimiexe/Desktop/DATAsets'
path_to_ready_csv = path_to_dataset_folder + '/DATA/CSVs'
path_encoded_csv = path_to_dataset_folder + '/DATA/CSVs_reverse_encoded'

In [ ]:
file = "OnlineNewsPopularity.csv"
df = pd.read_table(path_to_ready_csv + '/'+ file,sep=',')df.iloc[120:128,12:18]

In [ ]:
types = df.dtypes
cardi = df.apply(pd.Series.nunique)
df2=pd.DataFrame({'Types': types,
                  'Cardinality': cardi})
df2.head(10)

In [ ]:
#This is used to turn int-encoded features into str-encoded ones. This is desired
#because the str type is automatically recognised as categorical (enum) by H2O later.
#Set the variable 'features' to contain the int-encoded features to transform.
features = ["target_class"]
for target_feature in features:
    value_df = df[target_feature].value_counts(dropna=True).rename_axis('unique_values',axis=0).reset_index(name='value_counts',drop=False)
    value_df.sort_values(by='unique_values',inplace=True)
    value_df.reset_index(drop=True,inplace=True)
    lim = value_df.shape[0]
    replace_dict = {}
    for i in range(lim):
        replace_dict[ value_df.loc[i,'unique_values'] ] = 'Level_'+str(i)
    replace_dict = {target_feature : replace_dict}

    df.replace(to_replace = replace_dict, inplace=True, value=None)
df.head(10)

In [ ]:
#Used to REVERSE ONE HOT ENCODING of the "invOneHotFeatures" CONSECUTIVE-features' vector,
#creating a new column with the name given in ~~~HERE_1~~~. A check is made if every feature
#belongs to exactly one category. If not, this is not usable, and that case needs to be dealt
#with manually. The values of the new column can be default (categoricals) if "autoValues" == 1,
#or manually defined IN THE CODE (~~~HERE_2~~~) for autoValues == 0.

invOneHotFeatures = list(range(12,18)) #Consecutive feature indices
#invOneHotFeatures = [2,8,12] #Separate features indices
autoValues = 0
df2=df.iloc[:,invOneHotFeatures]
ser = df2.sum(axis=1)
if len(ser[ser != 0]) == df.shape[0]:
    if ser[ser!=1].sum() == 0:
        reverse_enc = []
        for x in range(df.shape[0]):
            for feat in invOneHotFeatures:
                if df.iloc[x,feat] == 1:
                    if autoValues == 0:
                        #~~~HERE_2~~~
                        #Manual definition of values in the new column.
                        reverse_enc.append(df.columns.values[feat].split('_')[-1])
                    elif autoValues == 1:
                        #Automated value assignement as "value_0, value_1" etc.
                        reverse_enc.append('value_'+str(feat-min(invOneHotFeatures)))
                    break
        #~~~HERE_1~~~
        df=df.assign(type_of_channel=reverse_enc)
        df.drop(df.columns.values[invOneHotFeatures],axis=1,inplace=True)
        print(df.head(10))
    else:
        print('Not all features belong exclusively to one category!')
else:
    print(str(len(ser[ser == 0])) + ' out of '+ str(df.shape[0]) +' features ('+ str(round(len(ser[ser == 0])/df.shape[0]*10000)/100) +'%) belong to NO category!')

In [ ]:
#Used to drop problematic rows detected above.
df2=df.iloc[:,invOneHotFeatures]
ser = df2.sum(axis=1)
df.drop(ser[ser == 0].index,axis=0,inplace=True)

In [ ]:
df.to_csv(path_encoded_csv + '/'+ file,index= False)